In [ ]:
import pandas as pd
from tqdm import tqdm
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm import tqdm
np.random.seed(42)
tf.random.set_seed(42)
from keras.callbacks import EarlyStopping 
from keras.models import load_model

In [ ]:
def training_dataset_(
    file_path,
    image_per_batch,
    start_row,
    ):
    df = pd.read_csv(file_path)[start_row:start_row+image_per_batch]
    
    df = df.drop('Unnamed: 0',axis=1)
    
    y = get_data_from_local(df)
    x = df.drop('image',axis=1)
    x = x.values
    
    return x , y

In [ ]:
def get_data_from_local(X_train):
    X_data =[]
    for file in tqdm(X_train['image']):
        face = cv2.imread(file,cv2.IMREAD_COLOR)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face =cv2.resize(face, (200,200))
#         face = np.expand_dims(face, axis=2)
        face = np.asarray(face)
        face = face.astype('float32')
        face /= 255
        X_data.append(face)
    return np.array(X_data)

### model

In [ ]:
from keras.layers import Conv2D , MaxPool2D ,ZeroPadding2D , UpSampling2D ,Reshape, Input ,Dense, Activation,Flatten
from keras.utils import np_utils
from keras.models import Model

def model(number_of_label):
    inp = Input(shape = (number_of_label,))
    
    fc1 = Dense(500,activation="sigmoid")(inp)
    
    fc2 = Dense(2000,activation="sigmoid")(fc1)
    
    fc3 = Dense(4000,activation="sigmoid")(fc2)

    fc4 = Dense(8836,activation="sigmoid")(fc3)

    res1 = Reshape((47,47,4))(fc4)
    zrp1 = ZeroPadding2D((1,1))(res1)

    conv4 = Conv2D(16,(3,3),padding="same" , activation ="relu")(zrp1)
    ups1 = UpSampling2D((2,2))(conv4)
    
    conv8 = Conv2D(16,(3,3),padding="same" , activation ="relu")(ups1)
    ups7 = UpSampling2D((2,2))(conv8)

    zrp2 = ZeroPadding2D((1,1))(ups7)
    conv5 = Conv2D(32,(3,3),padding="same" , activation ="relu")(zrp2)

    zrp3 = ZeroPadding2D((1,1))(conv5)
    conv6 = Conv2D(3,(3,3),padding="same" , activation="relu")(zrp3)
    c_a_e = Model(inputs=inp, outputs=conv6)
    c_a_e.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    c_a_e.summary()
    return c_a_e

### training

In [ ]:
def training(
    start_row,
    number_of_labels,
    file_path,
    image_per_batch,
    max_limit,
    path_of_model
):

    checkpoint = ModelCheckpoint(filepath=path_of_model,
                             monitor='val_accuracy',
                             save_format='tf',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )
    early_stopping_monitor = EarlyStopping(patience=2,verbose=1) 
    
    for x in range(start_row,max_limit,image_per_batch):
        
        print()
        print('**********{} -- {} out of {}************'.format(x,x+image_per_batch,max_limit))
        print()
    
        X_train , y_train   =   training_dataset_(
                        file_path = file_path,
                        start_row = x,
                        image_per_batch = image_per_batch,
        )
        
#         if x != 0:
        final_cnn = load_model(path_of_model)
#         else:
#             final_cnn = model(number_of_labels)
            
        try:
            final_cnn_history = final_cnn.fit(x=X_train,
                                          y=y_train,
                                          batch_size=64,
                                          validation_split=0.5,
                                          epochs=15,
                                          callbacks=[checkpoint,early_stopping_monitor],
                                          shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                                         )
        except:
            continue

In [ ]:
training(
    start_row = 16000,
    number_of_labels = 6,
    file_path = 'main_dataset1.csv',
    image_per_batch = 1000,
    max_limit = 35000,
    path_of_model = 'final_cnn_checkpoint.tf'
)

In [ ]:
import matplotlib.pyplot as plt
def drawimage(image,title="facial feature"):
    image = image.reshape(100,100,3)
    plt.imshow(image)
    plt.axis("off")
    plt.title(title,fontsize=20)
    plt.show()
model = load_model('./final_cnn_checkpoint.tf')
inp = np.array([1,1,1,1,1,1])
inp = np.expand_dims(inp,axis=0)
pred = model.predict(inp)
print(pred.shape)
drawimage(pred) 